#Chatbot Overview

This chatbot is designed to answer questions about return and refund policies for an e-commerce business. It uses a Retrieval-Augmented Generation (RAG) framework with the following components:

- **FAISS + MiniLM** to search the return policy document
- **OpenAI GPT-3.5** to generate a natural-language response
- **Gradio** to provide a user interface

The chatbot retrieves relevant text from the policy and uses it to generate a clear and helpful answer. It can answer a variety of user queries and shows the reference document used to support its response.


#Step 1: Load or Prepare Knowledge Base (KB)

In [1]:
import pandas as pd

data = {
    "title": ["Return Timeframe", "Return Conditions", "Refund Process", "Non-returnable Items"],
    "content": [
        "Customers may return products within 30 days of purchase with a valid receipt.",
        "Items must be unused and in original packaging. Clothing must have all tags attached.",
        "Refunds are issued to the original payment method within 7 business days after return approval.",
        "Final sale items, perishable goods, and personalized products are non-returnable."
    ]
}

kb_df = pd.DataFrame(data)
kb_df.head()


,title,content
0,Return Timeframe,Customers may return products within 30 days o...
1,Return Conditions,Items must be unused and in original packaging...
2,Refund Process,Refunds are issued to the original payment met...
3,Non-returnable Items,"Final sale items, perishable goods, and person..."


#Step 2: Create Embeddings and Build FAISS Index

In [2]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 21.6 MB/s eta 0:00:00


In [3]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Create embeddings
model = SentenceTransformer("all-MiniLM-L6-v2")
kb_embeddings = model.encode(kb_df["content"].tolist())

# FAISS index
dimension = kb_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(kb_embeddings))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

#Step 3: Define Prompt Template and LLM Parameters

In [4]:
system_prompt = (
    "You are a helpful customer support assistant that answers return policy questions. "
    "Keep your responses short, clear, and always based on the return-policy context provided. "
    "If you are unsure, say so clearly. Do not make up information."
)

##Set Parameter

In [5]:
TEMPERATURE = 0.3
TOP_P = 0.9


#Step 4: Query Handler Function with OpenAI API

In [6]:
import openai
from openai import OpenAI
import os

openai.api_key = "OPENAI_API_KEY"

def get_answer(query):
    query_vec = model.encode([query])
    top_k = 1
    D, I = index.search(np.array(query_vec), top_k)

    context = kb_df.iloc[I[0][0]]["content"]
    full_prompt = f"{system_prompt}\n\nContext: {context}\n\nUser Query: {query}"

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        temperature=TEMPERATURE,
        top_p=TOP_P,
        messages=[{"role": "system", "content": system_prompt},
                  {"role": "user", "content": full_prompt}]
    )

    return response.choices[0].message["content"], context


    from openai import OpenAI

client = OpenAI(api_key="OPENAI_API_KEY")

def get_answer(query):
    query_vec = model.encode([query])
    top_k = 1
    D, I = index.search(np.array(query_vec), top_k)

    context = kb_df.iloc[I[0][0]]["content"]

    full_prompt = f"{system_prompt}\n\nContext: {context}\n\nUser Query: {query}"

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        temperature=TEMPERATURE,
        top_p=TOP_P,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": full_prompt}
        ]
    )

    return response.choices[0].message.content, context



#Step 5: Gradio UI

In [7]:
import gradio as gr

def chatbot_interface(query):
    answer, context = get_answer(query)
    return f"Answer: {answer}\n\n(Reference: {context})"

gr.Interface(fn=chatbot_interface,
             inputs="text",
             outputs="text",
             title="Returns Support Chatbot").launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2b5a4f050aa603d812.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [8]:
test_queries = [
    "Can I return a product after 40 days?",
    "Do I need to keep the original receipt?",
    "What’s your refund process for damaged items?",
    "How do I exchange something if the size is wrong?"
]

for query in test_queries:
    response, context = get_answer(query)
    print(f"User: {query}")
    print(f"Answer: {response}")
    print(f"(Reference: {context})")
    print("-" * 80)


User: Can I return a product after 40 days?
Answer: I'm sorry, but our return policy allows returns within 30 days of purchase.
(Reference: Customers may return products within 30 days of purchase with a valid receipt.)
--------------------------------------------------------------------------------
User: Do I need to keep the original receipt?
Answer: Yes, you need to keep the original receipt for returns.
(Reference: Customers may return products within 30 days of purchase with a valid receipt.)
--------------------------------------------------------------------------------
User: What’s your refund process for damaged items?
Answer: Refunds for damaged items are issued to the original payment method within 7 business days after return approval.
(Reference: Refunds are issued to the original payment method within 7 business days after return approval.)
--------------------------------------------------------------------------------
User: How do I exchange something if the size is wro